# EquiCity Game Master Dashboard

## Introduction

To assist the game master in assessing the gameplay process, we put forth a set of statistical workflows that provide insight into how the players are making decisions, how effective these decisions are, and finally, if the decisions are improving during the gameplay. For this, we restate these questions as statistical hypotheses that their truth should be inferred.

* H1: Decision Homogeneity: Decisions are not homogenous across actors, w.r.t. sites, and colors.
* H2: Decision Improvement: Decisions are improving throughout the rounds.
* H3: Longer rounds will yield a more significant improvement in the decisions.

Given that in each round, each actor makes a decision about how much space in each site is allocated to each color, the decisions are formated as $\mathbf{X}_{[t,m,n,o]}$ where $t$ is the number of rounds, $m$ is the number of actors, $n$ is the number of sites, $o$ is the number of colors. Furthermore, since scores are calculated in each round per each criterion (three criteria in addition to individual criteria), the scores are formated as $\mathbf{O}_{[t,m+3]}$.

To infer the truth value of each of these hypotheses, we propose the following methods:
* M1: Given the decisions in $\mathbf{X}$, we use three-way classic ANOVA to compare the variance of the decision between Actors, Sites, and Colors. We propose a 0.05 alpha for the p-value. 
* M2: For the second hypothesis, we test the difference in one group of scores in two consecutive rounds by T-test.
* M3: For H3, we compute the Pearson correlation of the change in each round's scores (including group and individual scores) with the time spent on that round.

## Preparations

### Importing packages, methods and data 

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import pingouin as pg
from scipy.stats import ttest_1samp

/Users/shervinazadi/opt/miniconda3/envs/equicity/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# read all the backups
address = "../data/equicity_kabeldistrict/backup"
game_backups = os.listdir(address)
game_backups.pop(game_backups.index('.DS_Store'))
# sort the folders in ascending order
game_backups.sort()

In [3]:
game_id = -1
# load project and users dict
project_dict = pickle.load( open( address + "/" + game_backups[game_id] + "/project.pickle", "rb" ) )
users_dict = pickle.load( open( address + "/" + game_backups[game_id] + "/users.pickle", "rb" ) )

In [4]:
def print_keys(d, l=0):
    # iterate over the dict keys
    for k in d.keys():
        # extract the shape if the value is a list
        shape = np.array(d[k]).shape if isinstance(d[k],list) else ""
        # print info
        print(l*"  ", k, ":", type(d[k]), shape)
        # recurse if the value is a dict
        if isinstance(d[k],dict):
            print_keys(d[k], l+1)

### Prepare the decision matrix and dataframe

In [5]:
# extract the decision dictionary
decision_dict = project_dict["gameplay"]["X"]
# remove the 0 time stamp
_ = decision_dict.pop("0", None)
# extract and sort time stamps of decisions
dec_ts_sorted = np.array(list(decision_dict.keys()), dtype=int)
dec_ts_sorted.sort()
# extract a sample decision
x0 = np.array(decision_dict[str(dec_ts_sorted[0])])
# construct a 4 dimensional tensor containting all the observations:
# (Rounds, Actors, Sites, Colors)
X = np.array([decision_dict[str(ts)] for ts in dec_ts_sorted])

In [6]:
# convert the X to pandas dataframe
X_ind = np.indices(X.shape)
X_ind_flat = X_ind.reshape(X_ind.shape[0], -1)
X_df = pd.DataFrame(X_ind_flat.T, columns=["Round", "Actor", "Site", "Color"])
X_df["x"] = X.ravel()
X_df

,Round,Actor,Site,Color,x
0,0,0,0,0,0.152381
1,0,0,0,1,0.390476
2,0,0,0,2,0.200000
3,0,0,0,3,0.200000
4,0,0,0,4,0.057143
...,...,...,...,...,...
520,2,4,6,0,0.152381
521,2,4,6,1,0.104762
522,2,4,6,2,0.390476
523,2,4,6,3,0.247619


### Prepare the score matrix and dataframe

In [7]:
# extract the scores
score_dict = project_dict["game_info"]["scores"]
# remove the recent scores
[score_dict.pop(k, None) for k in ['change_score_t',  'closeness_score_t', 'environmental_score_t', 'individual_score_t']]
# extract individual scores
indiv_score = score_dict.pop("individual_score", None)
# separate valid scores
valid_scores = {}
for k, v in indiv_score.items():
    if isinstance(v, dict):
        valid_scores[k[:7]] = v
# update the score dictionary with the calid scores
score_dict.update(valid_scores)

# simplify the timestamps to unify
unified_score_dict = {}
for k, v in score_dict.items():
    local_score_dict = {}
    for k0, v0 in v.items():
        local_score_dict[str(int(int(k0)*1.e-5))] = v0
    unified_score_dict[k] = local_score_dict

# convert the data dictionary 
scores_df = pd.DataFrame(unified_score_dict).drop("0")
scores_df

,change_score,closeness_score,environmental_score,2rH45YZ,9gDjubf,Ccmr7hX,LNYUhpk,gHqMtuk
16383670,0.679600,0.314523,0.850648,0.172787,0.284601,0.330133,0.418206,0.347863
16383681,0.678689,0.308280,0.850327,0.171625,0.285477,0.329160,0.417075,0.347863
16383706,0.655034,0.299094,0.824199,0.166619,0.286388,0.325213,0.408819,0.347863


## Analysis Implementation

### H1
**Decisions are not homgenous across actors, w.r.t. sites, and colors.**

In [8]:
# three-way ANOVA usign Pingouin library
aov = pg.anova(data=X_df, dv='x', between=['Actor', 'Site', 'Color'], detailed=True)
aov

,Source,SS,DF,MS,F,p-unc,np2
0,Actor,3.417689e-29,4.0,8.544223e-30,2.392021e-27,1.000000e+00,2.733738e-29
1,Site,4.029791e-29,6.0,6.716318e-30,1.880285e-27,1.000000e+00,3.223345e-29
2,Color,6.040384e-01,4.0,1.510096e-01,4.227630e+01,6.393050e-29,3.257629e-01
3,Actor * Site,5.117565e-29,24.0,2.132319e-30,5.969590e-28,1.000000e+00,4.093433e-29
4,Actor * Color,2.834467e+00,16.0,1.771542e-01,4.959568e+01,9.610746e-80,6.939304e-01
5,Site * Color,1.179786e+00,24.0,4.915776e-02,1.376209e+01,1.998796e-37,4.855137e-01
6,Actor * Site * Color,2.766440e+00,96.0,2.881708e-02,8.067563e+00,2.114385e-48,6.887467e-01
7,Residual,1.250189e+00,350.0,3.571968e-03,NaN,NaN,NaN


### H2
**Decision Improvement: Decsisons are improving throughout the rounds.**

In [9]:
# iterate over the rounds
for i in range(len(scores_df) - 1):
    # extract the scores of the rounds
    s0 = scores_df.iloc[i].to_numpy()
    s1 = scores_df.iloc[i+1].to_numpy()
    # run the t test
    obs = ttest_1samp(s1 - s0 , 0)
    print("round :", i)
    print("statistic :", obs.statistic)
    print("p-value :", obs.pvalue)

round : 0
statistic : -1.6288181349765882
p-value : 0.14737518610312203
round : 1
statistic : -2.609286812560844
p-value : 0.034949586466993886


### H3
**Longer rounds will yield a greater improvement in the decisions.**

In [10]:
# extract the index as time
scores_df["time"] = scores_df.index
# convert the type from string to int
scores_df = scores_df.astype({'time': 'int'})
# compute the difference between rounds of the game
scores_diff = scores_df.diff()
# drop the NaN row
scores_diff = scores_diff.dropna()

# compute the pearson correlation
scores_diff.corr()

,change_score,closeness_score,environmental_score,2rH45YZ,9gDjubf,Ccmr7hX,LNYUhpk,gHqMtuk,time
change_score,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
closeness_score,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
environmental_score,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
2rH45YZ,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
9gDjubf,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0
Ccmr7hX,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
LNYUhpk,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
gHqMtuk,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0
time,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0
